# Pré-requis

In [ ]:
!pip install google-generativeai pypdf sentence-transformers faiss-cpu numpy

# Extraction PDF

### pip install

In [12]:
!pip install --upgrade pdf2image pytesseract

  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for pytesseract from https://files.pythonhosted.org/packages/7a/33/8312d7ce74670c9d39a532b2c246a853861120486be9443eebf048043637/pytesseract-0.3.13-py3-none-any.whl.metadata


### travail

In [3]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [4]:
old_doc = extract_text_from_pdf("ancien_clean.pdf")
new_doc = extract_text_from_pdf("nouveau.pdf")

print(old_doc[:1000])
print("#################")
print(new_doc[:1000])

Gynécologie
Obstétrique
Front matterChez le même éditeur
Dans la même collection
Activité physique et sportive : facteur de santé, par le Collège français des enseignants en médecine et trauma-
tologie du sport et de l'exercice physique (CFEMTSEP), 2019, 96 pages.
Anatomie et cytologie pathologiques, par le Collège français des pathologistes (CoPath), 3 e édition, 2019, 
416 pages.
Chirurgie maxillo-faciale et stomatologie, par le Collège hospitalo-universitaire français de chirurgie maxillofa -
ciale et stomatologie, 5e édition, 2021, 432 pages.
Dermatologie, par le Collège des enseignants en dermatologie de France (CEDEF), 7e édition, 2017, 472 pages.
Endocrinologie, diabétologie et maladies métaboliques, par le Collège des enseignants d'endocrinologie, dia -
bète et maladies métaboliques (CEEDMM), 5e édition, 2021, 568 pages.
Gériatrie, par le Collège national des enseignants de gériatrie (CNEG), 5e édition, 2021, 400 pages.
Gynécologie obstétrique, par le Collège national des gynéc

### saves

In [12]:
with open("saves/ancien.txt", "w") as f:
    f.write(old_doc)
with open("saves/nouveau.txt", "w") as f:
    f.write(new_doc)

# Splitting en chunks

### pip install

In [8]:
!pip install langchain tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/6f/07/c67ad1724b8e14e2b4c8cca04b15da158733ac60136879131db05dda7c30/tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/893.9 kB ? eta -:--:--
   --- ----------------------------------- 71.7/893.9 kB 975.2 kB/s eta 0:00:01
   ----------------- ---------------------- 399.4/893.9 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 893.9/893.9 kB 6.3 MB/s eta 0:00:00


### travail

In [9]:
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text_into_chunks(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    chunks = []
    # Une approche simple par caractères, mais vous pouvez améliorer avec un split par paragraphe ou phrase
    words = text.split()
    current_chunk = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 <= chunk_size:
            current_chunk.append(word)
            current_length += len(word) + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:] + [word] # Garder un chevauchement
            current_length = sum(len(w) + 1 for w in current_chunk)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def split_text_langchain(text: str, chunk_size: int = 250, overlap: int = 25) -> List[str]:
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
    )
    return text_splitter.split_text(text)

In [10]:
chunks_old = split_text_langchain(old_doc)
chunks_new = split_text_langchain(new_doc)

print(f"Nombre de chunks dans l'ancien document: {len(chunks_old)}")
print(f"Premier chunk de l'ancien document: {chunks_old[0]}")
print(f"Nombre de chunks dans le nouveau document: {len(chunks_new)}")
print(f"Premier chunk du nouveau document: {chunks_new[0]}")

Nombre de chunks dans l'ancien document: 3295
Premier chunk de l'ancien document: Gynécologie
Obstétrique
Front matterChez le même éditeur
Dans la même collection
Activité physique et sportive : facteur de santé, par le Collège français des enseignants en médecine et trauma-
tologie du sport et de l'exercice physique (CFEMTSEP), 2019, 96 pages.
Anatomie et cytologie pathologiques, par le Collège français des pathologistes (CoPath), 3 e édition, 2019, 
416 pages.
Chirurgie maxillo-faciale et stomatologie, par le Collège hospitalo-universitaire français de chirurgie maxillofa -
ciale et stomatologie, 5e édition, 2021, 432 pages.
Dermatologie, par le Collège des enseignants en dermatologie de France (CEDEF), 7e édition, 2017, 472 pages.
Nombre de chunks dans le nouveau document: 3279
Premier chunk du nouveau document: Gynécologie 
ObstétriqueGynécologie 
Obstétrique
Sous l’égide du Collège National des Gynécologues et Obstétriciens Français 
et du Collège des Enseignants de Gynécologie-Ob

### saves

In [13]:
import pickle

with open("saves/chunks/ancien_chunks.pkl", "wb") as f:
    pickle.dump(chunks_old, f)
with open("saves/chunks/nouveau_chunks.pkl", "wb") as f:
    pickle.dump(chunks_new, f)

# Encoding & Vectorisation

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Charger un modèle d'embedding
# 'all-MiniLM-L6-v2' est un bon compromis taille/performance pour le texte général.
# Pour des cas plus spécifiques, vous pourriez envisager des modèles plus grands.
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embeddings(texts: List[str]):
    return model.encode(texts, convert_to_tensor=True).cpu().numpy()

# Exemple d'utilisation
# embeddings1 = get_embeddings(chunks1)
# embeddings2 = get_embeddings(chunks2)

# Créer des index FAISS pour chaque document
# dimension_embedding = embeddings1.shape[1]
# index1 = faiss.IndexFlatL2(dimension_embedding)
# index1.add(embeddings1)

# index2 = faiss.IndexFlatL2(dimension_embedding)
# index2.add(embeddings2)

# Comparaison & identification des différences

In [ ]:
def find_differences(chunks1: List[str], embeddings1: np.ndarray, index1: faiss.IndexFlatL2,
                     chunks2: List[str], embeddings2: np.ndarray, index2: faiss.IndexFlatL2,
                     similarity_threshold: float = 0.8) -> List[str]:
    differences = []

    # Parcourir les chunks du document 2 et chercher leur correspondance dans le document 1
    for i, chunk2 in enumerate(chunks2):
        query_embedding = embeddings2[i].reshape(1, -1)
        distances, indices = index1.search(query_embedding, k=1) # Chercher le plus proche dans doc1

        closest_distance = distances[0][0]
        closest_chunk1_index = indices[0][0]

        # Si la distance est grande (faible similarité), ou si le chunk n'est pas "trouvé" (distance > seuil),
        # cela indique une possible différence ou un ajout.
        # Note: L2 distance plus faible = plus similaire.
        # Vous devrez ajuster le seuil en fonction de vos observations.
        if closest_distance > (1 - similarity_threshold): # Adapter la condition pour L2 distance
            differences.append(f"Ajout/Modification probable dans Doc2: '{chunk2}' (Distance au plus proche de Doc1: {closest_distance:.4f})")
        else:
            # Pour des modifications subtiles, vous pouvez aussi comparer le chunk original avec le chunk trouvé
            # si la similarité est juste au-dessus du seuil mais pas parfaite.
            # Cela nécessiterait une étape supplémentaire d'analyse.
            pass # Ici, le chunk est considéré comme similaire

    # Vous pouvez aussi faire l'inverse (chercher les différences de doc1 par rapport à doc2)
    # selon votre besoin de détection des suppressions.

    return differences

In [ ]:
import difflib

def detailed_chunk_diff(chunk1: str, chunk2: str) -> List[str]:
    d = difflib.Differ()
    diff = list(d.compare(chunk1.splitlines(), chunk2.splitlines()))
    return diff

# Agent synthétisation différences

In [ ]:
import google.generativeai as genai
import os

# Configurez votre clé API Gemini
# genai.configure(api_key=os.environ["GOOGLE_API_KEY"]) # Ou directement genai.configure(api_key="YOUR_API_KEY")

def summarize_differences_with_gemini(differences: List[str]):
    if not differences:
        return "Aucune différence significative détectée entre les documents."

    # Joindre les différences pour les présenter au modèle
    differences_text = "\n".join(differences)

    prompt = f"""
    Voici une liste de différences potentielles détectées entre deux documents.
    Veuillez analyser ces différences et fournir une liste claire et concise de chaque ajout ou modification significative,
    en expliquant brièvement de quoi il s'agit.

    Différences détectées :
    {differences_text}

    Liste des ajouts/modifications:
    """

    # Utilisez Gemini 1.5 Flash
    model = genai.GenerativeModel('gemini-1.5-flash-latest')

    # Ajustez max_output_tokens si vos résumés sont très longs
    response = model.generate_content(prompt, generation_config={"max_output_tokens": 800})

    try:
        return response.text
    except ValueError as e:
        print(f"Erreur lors de la génération de la réponse : {e}")
        print(f"Prompt: {prompt}")
        print(f"Candidats de réponse: {response.candidates}")
        return "Impossible de générer le résumé des différences."

# MAIN

In [ ]:
# Mettez vos chemins de fichiers PDF ici
PDF_DOC1 = "document1.pdf"
PDF_DOC2 = "document2.pdf"

# 1. Extraction du texte
doc1_text = extract_text_from_pdf(PDF_DOC1)
doc2_text = extract_text_from_pdf(PDF_DOC2)

# 2. Division en chunks
chunks1 = split_text_into_chunks(doc1_text, chunk_size=300, overlap=30) # Ajustez les tailles de chunk
chunks2 = split_text_into_chunks(doc2_text, chunk_size=300, overlap=30)

print(f"Document 1 a {len(chunks1)} chunks.")
print(f"Document 2 a {len(chunks2)} chunks.")

# 3. Embedding et vectorisation
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings1 = get_embeddings(chunks1)
embeddings2 = get_embeddings(chunks2)

dimension_embedding = embeddings1.shape[1]
index1 = faiss.IndexFlatL2(dimension_embedding)
index1.add(embeddings1)
index2 = faiss.IndexFlatL2(dimension_embedding)
index2.add(embeddings2)


# 4. Identification des différences
# Seuils: la similarité cosinus va de -1 à 1. Une valeur de 0.8 signifie 80% de similarité.
# Pour la distance L2, une valeur plus petite signifie plus similaire. 0 étant identique.
# Si vous avez converti votre similarité cosinus en distance L2 (distance = 1 - similarité),
# alors un seuil de 0.2 (1-0.8) serait approprié.
# Vous devrez expérimenter avec ce seuil.
potential_differences = find_differences(chunks1, embeddings1, index1,
                                         chunks2, embeddings2, index2,
                                         similarity_threshold=0.85)

print(f"Nombre de différences potentielles détectées : {len(potential_differences)}")
# print("\n".join(potential_differences[:5])) # Affiche les 5 premières pour un aperçu

# 5. Synthèse avec Gemini
# Assurez-vous que votre clé API est configurée
# import os
# os.environ["GOOGLE_API_KEY"] = "VOTRE_CLE_API" # REMPLACEZ PAR VOTRE VRAIE CLE API
# genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

if potential_differences:
    final_summary = summarize_differences_with_gemini(potential_differences)
    print("\nRésumé des différences par Gemini :")
    print(final_summary)
else:
    print("Aucune différence significative trouvée nécessitant une synthèse par Gemini.")